# 1. train/test 나눠서 한거

## 1. train from "DROPEDGE" - about 1000 train sets

In [5]:
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

### 이거는 저장해두자. 어떻게 test_mask 안 겹치게 뽑는지.

In [51]:
dataset = Planetoid(root='data/', name='Cora')

# 데이터 확인
data = dataset[0]  # Cora 데이터셋은 하나의 그래프 데이터로 구성됨


In [34]:
test_mask = data.test_mask
test_mask.sum()

tensor(1000)

In [52]:
test_mask = data.test_mask

# 3. test_mask가 False인 노드들(= non-test 노드)만 골라낸 인덱스
non_test_indices = torch.where(~test_mask)[0]  # ~test_mask: test_mask가 False인 노드

# 4. non_test_indices 중에서 랜덤하게 1000개 선택
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1200]]

# 5. 새로운 train_mask 생성
new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True

# 6. data.train_mask에 할당
data.train_mask = new_train_mask

# 이제 data.train_mask에는 test set과 겹치지 않는 1000개의 노드만 True로 설정됨
print("Train set size:", data.train_mask.sum().item())
print("Overlap with test set:", (data.train_mask & data.test_mask).sum().item())  # 0이어야 정상


Train set size: 1200
Overlap with test set: 0


In [53]:
data.train_mask.sum()

tensor(1200)

In [54]:

# 데이터 확인
#data = dataset[0]  # Cora 데이터셋은 하나의 그래프 데이터로 구성됨

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data, selected_nodes=None):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        x = F.log_softmax(x, dim=1)

        # 선택된 노드만 반환
        if selected_nodes is not None and len(selected_nodes) > 0:
            selected_nodes = torch.tensor(selected_nodes, dtype=torch.long, device=x.device)
            return x[selected_nodes]  
        return x  

# 모델 및 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss()

# Poisson 시뮬레이션 + 학습
N = data.num_nodes  # 노드 개수
lambda_ = 0.5  # Poisson rate
T = 20  # 총 시뮬레이션 시간

# 노드별 첫 번째 이벤트 발생 시간 (지수 분포 샘플링)
event_times = [np.random.exponential(1/lambda_) for _ in range(N)]
event_logs = [[] for _ in range(N)]  # 각 노드의 이벤트 발생 시간 기록

# 훈련 데이터 마스크 가져오기
train_mask = data.train_mask.cpu().numpy()

# **Training 함수 (선택된 노드만 학습)**
def train(selected_nodes):
    if len(selected_nodes) == 0:  # 선택된 노드가 없으면 학습 건너뛰기
        return None
    
    model.train()
    optimizer.zero_grad()

    out = model(data.to(device), selected_nodes)  # 선택된 노드만 예측
    labels = data.y[selected_nodes].to(device)  # 선택된 노드들의 레이블만 사용

    loss = criterion(out, labels)  # 선택된 노드들에 대해서만 loss 계산
    loss.backward()
    optimizer.step()

    return loss.item()

# **시뮬레이션 실행 + 학습**
t = 0
while t < T:
    selected_nodes = []  # 해당 노드들만 Convolution 하기 위해 index 선택
    for i in range(N):
        if t >= event_times[i] and train_mask[i]:  # Poisson clock이 울리고 train_mask가 True인 경우만 선택
            selected_nodes.append(i)  # 해당 노드들만 선택
            event_logs[i].append(t)  # 발생 시간 저장
            event_times[i] += np.random.exponential(1/lambda_)  # 다음 이벤트 시간 설정
            
    t += 0.1  # 작은 타임스텝으로 증가
    if selected_nodes:
        selected = torch.tensor(selected_nodes, dtype=torch.long, device=device)
        loss = train(selected)
        if loss is not None:
            print(f"Time {t:.1f}: Loss = {loss:.4f}")


/tmp/ipykernel_2354475/1954964068.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  selected_nodes = torch.tensor(selected_nodes, dtype=torch.long, device=x.device)


Time 0.2: Loss = 1.9479
Time 0.3: Loss = 1.8857
Time 0.4: Loss = 1.7712
Time 0.5: Loss = 1.7585
Time 0.6: Loss = 1.5070
Time 0.7: Loss = 1.5719
Time 0.8: Loss = 1.5061
Time 0.9: Loss = 1.3534
Time 1.0: Loss = 1.3661
Time 1.1: Loss = 1.4134
Time 1.2: Loss = 1.2804
Time 1.3: Loss = 1.2186
Time 1.4: Loss = 1.1115
Time 1.5: Loss = 1.0350
Time 1.6: Loss = 1.0409
Time 1.7: Loss = 0.8810
Time 1.8: Loss = 0.9083
Time 1.9: Loss = 0.7292
Time 2.0: Loss = 0.7991
Time 2.1: Loss = 0.8788
Time 2.2: Loss = 0.7259
Time 2.3: Loss = 0.7311
Time 2.4: Loss = 0.5960
Time 2.5: Loss = 0.6093
Time 2.6: Loss = 0.5774
Time 2.7: Loss = 0.5463
Time 2.8: Loss = 0.5751
Time 2.9: Loss = 0.4810
Time 3.0: Loss = 0.3837
Time 3.1: Loss = 0.4277
Time 3.2: Loss = 0.4005
Time 3.3: Loss = 0.3946
Time 3.4: Loss = 0.3397
Time 3.5: Loss = 0.4733
Time 3.6: Loss = 0.3217
Time 3.7: Loss = 0.3399
Time 3.8: Loss = 0.3028
Time 3.9: Loss = 0.4294
Time 4.0: Loss = 0.2548
Time 4.1: Loss = 0.3524
Time 4.2: Loss = 0.2278
Time 4.3: Loss =

In [55]:
def test():
    model.eval()
    with torch.no_grad():
        out = model(data.to(device))  # 모든 노드에 대한 예측
        pred = out.argmax(dim=1)  # 가장 높은 확률을 가진 클래스를 예측
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()  # 정확히 맞춘 개수
        acc = correct / data.test_mask.sum().item()  # 정확도 계산
    return acc

# 🔹 테스트 실행
test_acc = test()
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.8510


In [25]:
assert not any(data.test_mask[train_indices]), "Training set contains test nodes!"


AssertionError: Training set contains test nodes!

In [7]:
train_mask

array([ True,  True,  True, ...,  True,  True,  True])

In [88]:
dataset = Planetoid(root='data/', name='Citeseer')

# 데이터 확인
data = dataset[0]  # Cora 데이터셋은 하나의 그래프 데이터로 구성됨


In [89]:
test_mask = data.test_mask

# 3. test_mask가 False인 노드들(= non-test 노드)만 골라낸 인덱스
non_test_indices = torch.where(~test_mask)[0]  # ~test_mask: test_mask가 False인 노드

# 4. non_test_indices 중에서 랜덤하게 1000개 선택
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]

# 5. 새로운 train_mask 생성
new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True

# 6. data.train_mask에 할당
data.train_mask = new_train_mask

# 이제 data.train_mask에는 test set과 겹치지 않는 1000개의 노드만 True로 설정됨
print("Train set size:", data.train_mask.sum().item())
print("Overlap with test set:", (data.train_mask & data.test_mask).sum().item())  # 0이어야 정상


Train set size: 1800
Overlap with test set: 0


## 이게 진짜임. train_mask 설정시켜놓고(1200개) poisson process로 시뮬레이션 학습 후 200 epoch 돌리기.(Citeseer with 1800개)

In [3]:
dataset = Planetoid(root='data/', name='citeseer')
data = dataset[0]

#############################################
# 2) (선택) 원하는 방식으로 train_mask 설정
#    - 예: 테스트셋 제외 후 1000개 노드 골라서 train_mask 구성 등
#    - 여기서는 간단히 data.train_mask 사용
#############################################
test_mask = data.test_mask

# 3. test_mask가 False인 노드들(= non-test 노드)만 골라낸 인덱스
non_test_indices = torch.where(~test_mask)[0]  # ~test_mask: test_mask가 False인 노드

# 4. non_test_indices 중에서 랜덤하게 1000개 선택
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]

# 5. 새로운 train_mask 생성
new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True

# 6. data.train_mask에 할당
train_mask = new_train_mask

# 이제 data.train_mask에는 test set과 겹치지 않는 1000개의 노드만 True로 설정됨
print("Train set size:", train_mask.sum().item())
print("Overlap with test set:", (train_mask & data.test_mask).sum().item())  # 0이어야 정상

#############################################
# 3) GCN 모델 정의
#############################################
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data, selected_nodes=None):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.log_softmax(x, dim=1)

        # 특정 노드만 반환 (Poisson 시뮬레이션 시 사용)
        if selected_nodes is not None and len(selected_nodes) > 0:
            selected_nodes = torch.tensor(selected_nodes, dtype=torch.long, device=x.device)
            return x[selected_nodes]  
        return x

#############################################
# 4) 모델 및 옵티마이저 정의
#############################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss()

#############################################
# 5) (시나리오 B) 먼저 Poisson 시뮬레이션 기반 학습
#############################################
N = data.num_nodes
lambda_ = 0.5
T = 10  # 총 시뮬레이션 시간
time_step = 0.1

# 각 노드별 첫 이벤트 시간 (지수분포)
event_times = [np.random.exponential(1/lambda_) for _ in range(N)]
event_logs = [[] for _ in range(N)]

# numpy 형태의 train_mask (True/False)
train_mask_np = train_mask.cpu().numpy()

def train_poisson(selected_nodes):
    if len(selected_nodes) == 0:
        return None
    model.train()
    optimizer.zero_grad()

    out = model(data.to(device), selected_nodes)
    labels = data.y[selected_nodes].to(device)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# ===== Poisson 시뮬레이션 루프 =====
model.train()
t = 0
while t < T:
    selected_nodes = []
    for i in range(N):
        # Poisson 이벤트 발생 & train_mask
        if t >= event_times[i] and train_mask_np[i]:
            selected_nodes.append(i)
            event_logs[i].append(t)
            # 다음 이벤트 시간
            event_times[i] += np.random.exponential(1/lambda_)

    t += time_step

    # 학습
    if selected_nodes:
        loss_val = train_poisson(selected_nodes)
        if loss_val is not None:
            print(f"[Time {t:.1f}] Poisson Loss: {loss_val:.4f}")

print("=== Poisson-based Training Complete ===")


#############################################
# 6) 이제 200 epoch 학습(풀-배치 방식)
#############################################
model.train()
for epoch in range(1, 201):
    optimizer.zero_grad()
    out = model(data.to(device))  # 전체 노드에 대한 forward
    loss = criterion(out[train_mask], data.y[train_mask].to(device))
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss.item():.4f}")

print("=== Final 200 Epoch Training Complete ===")

def test():
    model.eval()
    with torch.no_grad():
        out = model(data.to(device))  # 모든 노드에 대한 예측
        pred = out.argmax(dim=1)  # 가장 높은 확률을 가진 클래스를 예측
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()  # 정확히 맞춘 개수
        acc = correct / data.test_mask.sum().item()  # 정확도 계산
    return acc

# 🔹 테스트 실행
test_acc = test()
print(f"Test Accuracy: {test_acc:.4f}")

Train set size: 1800
Overlap with test set: 0
[Time 0.2] Poisson Loss: 1.7926
[Time 0.3] Poisson Loss: 1.7413
[Time 0.4] Poisson Loss: 1.6850
[Time 0.5] Poisson Loss: 1.5921
[Time 0.6] Poisson Loss: 1.4736
[Time 0.7] Poisson Loss: 1.4071
[Time 0.8] Poisson Loss: 1.4375
[Time 0.9] Poisson Loss: 1.3083
[Time 1.0] Poisson Loss: 1.3243
[Time 1.1] Poisson Loss: 1.1727
[Time 1.2] Poisson Loss: 1.1228
[Time 1.3] Poisson Loss: 1.1177
[Time 1.4] Poisson Loss: 1.0256
[Time 1.5] Poisson Loss: 1.1375
[Time 1.6] Poisson Loss: 1.0177
[Time 1.7] Poisson Loss: 0.9198
[Time 1.8] Poisson Loss: 0.8954
[Time 1.9] Poisson Loss: 0.8644
[Time 2.0] Poisson Loss: 0.9075
[Time 2.1] Poisson Loss: 0.8778
[Time 2.2] Poisson Loss: 0.9053
[Time 2.3] Poisson Loss: 0.7421
[Time 2.4] Poisson Loss: 0.7104
[Time 2.5] Poisson Loss: 0.6942
[Time 2.6] Poisson Loss: 0.7112
[Time 2.7] Poisson Loss: 0.7482
[Time 2.8] Poisson Loss: 0.8174
[Time 2.9] Poisson Loss: 0.6477
[Time 3.0] Poisson Loss: 0.7243
[Time 3.1] Poisson Loss: 0

### 이거는 그냥 바닐라 GCN

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data

# 1. 데이터셋 로드 (Citeseer)
dataset = Planetoid(root='data/', name='Citeseer')
data = dataset[0]

# 2. Train Mask 생성 (test_mask와 겹치지 않는 노드 1800개 선택)
test_mask = data.test_mask
non_test_indices = torch.where(~test_mask)[0]  # test_mask가 False인 노드들
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]  # 1800개 샘플링

new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True
train_mask = new_train_mask

# 3. GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, in_feats, hidden_dim, out_feats):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_feats, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, out_feats)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# 4. 모델, 옵티마이저, 손실 함수 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCN(dataset.num_features, 16, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss()

# 5. 학습 함수 정의
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])  # 선택된 train_mask만 사용
    loss.backward()
    optimizer.step()
    return loss.item()

# 6. 테스트 함수 정의
@torch.no_grad()
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)  # 가장 높은 확률을 가진 클래스 선택

    accs = []
    for mask in [train_mask, data.val_mask, data.test_mask]:
        correct = (pred[mask] == data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)

    return accs  # [train_acc, val_acc, test_acc]

# 7. 학습 진행
best_test_acc = 0
best_epoch = 0

for epoch in range(200):
    loss = train()
    train_acc, val_acc, test_acc = test()

    # 🔹 Test Accuracy가 가장 높은 순간을 저장
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch  # 언제 최고였는지 저장

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")

print(f"Best Test Accuracy: {best_test_acc:.4f} (at Epoch {best_epoch})")


Epoch 000 | Loss: 1.7826 | Train Acc: 0.5711 | Val Acc: 0.5800 | Test Acc: 0.5270
Epoch 010 | Loss: 0.7221 | Train Acc: 0.8439 | Val Acc: 0.8580 | Test Acc: 0.7770
Epoch 020 | Loss: 0.4688 | Train Acc: 0.8783 | Val Acc: 0.8840 | Test Acc: 0.7760
Epoch 030 | Loss: 0.3683 | Train Acc: 0.9094 | Val Acc: 0.9080 | Test Acc: 0.7550
Epoch 040 | Loss: 0.3112 | Train Acc: 0.9300 | Val Acc: 0.9080 | Test Acc: 0.7530
Epoch 050 | Loss: 0.2975 | Train Acc: 0.9344 | Val Acc: 0.9120 | Test Acc: 0.7520
Epoch 060 | Loss: 0.2806 | Train Acc: 0.9400 | Val Acc: 0.9080 | Test Acc: 0.7550
Epoch 070 | Loss: 0.2648 | Train Acc: 0.9394 | Val Acc: 0.9100 | Test Acc: 0.7510
Epoch 080 | Loss: 0.2440 | Train Acc: 0.9544 | Val Acc: 0.9260 | Test Acc: 0.7440
Epoch 090 | Loss: 0.2425 | Train Acc: 0.9483 | Val Acc: 0.9120 | Test Acc: 0.7530
Epoch 100 | Loss: 0.2348 | Train Acc: 0.9483 | Val Acc: 0.9220 | Test Acc: 0.7460
Epoch 110 | Loss: 0.2253 | Train Acc: 0.9506 | Val Acc: 0.9180 | Test Acc: 0.7490
Epoch 120 | Loss

### 바닐라gcn 4-layers

In [128]:
import torch
import torch.nn.functional as F
import torch.nn.init as init
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# ✅ 1. Citeseer 데이터셋 로드
dataset = Planetoid(root='data/', name='Citeseer')
data = dataset[0]

# ✅ 2. Train Mask 생성 (test_mask와 겹치지 않는 노드 1800개 선택)
test_mask = data.test_mask
non_test_indices = torch.where(~test_mask)[0]
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]

new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True
train_mask = new_train_mask  # 새 train_mask 적용

# ✅ 3. 4-layer Vanilla GCN
class GCN(torch.nn.Module):
    def __init__(self, in_features, hidden_dim1, hidden_dim2, hidden_dim3, out_features):
        super().__init__()
        self.conv1 = GCNConv(in_features, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, hidden_dim3)
        self.conv4 = GCNConv(hidden_dim3, out_features)

        self.reset_parameters()

    def reset_parameters(self):
        init.xavier_uniform_(self.conv1.lin.weight)
        init.xavier_uniform_(self.conv2.lin.weight)
        init.xavier_uniform_(self.conv3.lin.weight)
        init.xavier_uniform_(self.conv4.lin.weight)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # ✅ 첫 번째 GCN 레이어
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # ✅ 두 번째 GCN 레이어
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # ✅ 세 번째 GCN 레이어
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # ✅ 네 번째 GCN 레이어 (출력)
        x = self.conv4(x, edge_index)

        return F.log_softmax(x, dim=1)

# ✅ 4. 모델 및 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(
    in_features=dataset.num_features, 
    hidden_dim1=64, 
    hidden_dim2=32, 
    hidden_dim3=16, 
    out_features=dataset.num_classes
).to(device)

data = data.to(device)
train_mask = train_mask.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
criterion = torch.nn.NLLLoss()

# ✅ 5. 학습 루프
best_test_acc = 0
best_epoch = 0

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # 테스트 수행
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
        test_acc = correct / data.test_mask.sum().item()

    # 🔹 Best Test Accuracy 저장
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch

    if epoch % 10 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")

print(f"=== Training Complete ===")
print(f"Best Test Accuracy: {best_test_acc:.4f} (at Epoch {best_epoch})")


[Epoch 000] Loss: 1.8004 | Test Acc: 0.3750
[Epoch 010] Loss: 1.1123 | Test Acc: 0.7600
[Epoch 020] Loss: 0.7755 | Test Acc: 0.7800
[Epoch 030] Loss: 0.5919 | Test Acc: 0.7670
[Epoch 040] Loss: 0.4823 | Test Acc: 0.7660
[Epoch 050] Loss: 0.4299 | Test Acc: 0.7620
[Epoch 060] Loss: 0.3899 | Test Acc: 0.7530
[Epoch 070] Loss: 0.3583 | Test Acc: 0.7600
[Epoch 080] Loss: 0.3632 | Test Acc: 0.7440
[Epoch 090] Loss: 0.3467 | Test Acc: 0.7400
[Epoch 100] Loss: 0.3461 | Test Acc: 0.7500
[Epoch 110] Loss: 0.3265 | Test Acc: 0.7460
[Epoch 120] Loss: 0.3368 | Test Acc: 0.7480
[Epoch 130] Loss: 0.3166 | Test Acc: 0.7460
[Epoch 140] Loss: 0.2938 | Test Acc: 0.7440
[Epoch 150] Loss: 0.2898 | Test Acc: 0.7380
[Epoch 160] Loss: 0.2917 | Test Acc: 0.7430
[Epoch 170] Loss: 0.2772 | Test Acc: 0.7300
[Epoch 180] Loss: 0.2791 | Test Acc: 0.7490
[Epoch 190] Loss: 0.2780 | Test Acc: 0.7410
=== Training Complete ===
Best Test Accuracy: 0.7890 (at Epoch 13)


### 이거 4-layers

In [5]:
import torch
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# ✅ 1. Citeseer 데이터셋 로드
dataset = Planetoid(root='data/', name='Citeseer')
data = dataset[0]

# ✅ 2. Train Mask 생성 (test_mask와 겹치지 않는 1800개 노드 선택)
test_mask = data.test_mask
non_test_indices = torch.where(~test_mask)[0]
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]

new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True
train_mask = new_train_mask  # 새 train_mask 적용

# ✅ 3. 4-layer GCN 모델 (Xavier 제거)
class GCN(torch.nn.Module):
    def __init__(self, in_features, hidden_dim1, hidden_dim2, hidden_dim3, out_features):
        super().__init__()
        self.conv1 = GCNConv(in_features, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, hidden_dim3)
        self.conv4 = GCNConv(hidden_dim3, out_features)

    def forward(self, data, selected_nodes=None):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)

        x = self.conv4(x, edge_index)
        x = F.log_softmax(x, dim=1)

        if selected_nodes is not None and len(selected_nodes) > 0:
            selected_nodes = torch.tensor(selected_nodes, dtype=torch.long, device=x.device)
            return x[selected_nodes]  
        return x

# ✅ 4. 모델 및 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(
    in_features=dataset.num_features, 
    hidden_dim1=64, 
    hidden_dim2=32, 
    hidden_dim3=16, 
    out_features=dataset.num_classes
).to(device)

data = data.to(device)
train_mask = train_mask.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
criterion = torch.nn.NLLLoss()

# ✅ 5. Poisson 시뮬레이션 기반 학습
N = data.num_nodes
lambda_ = 0.5
T = 6  # 총 시뮬레이션 시간
time_step = 0.1

# 각 노드별 첫 이벤트 시간 (지수분포)
event_times = [np.random.exponential(1/lambda_) for _ in range(N)]
event_logs = [[] for _ in range(N)]

# numpy 형태의 train_mask (True/False)
train_mask_np = train_mask.cpu().numpy()

def train_poisson(selected_nodes):
    if len(selected_nodes) == 0:
        return None
    model.train()
    optimizer.zero_grad()

    out = model(data.to(device), selected_nodes)
    labels = data.y[selected_nodes].to(device)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# ===== Poisson 시뮬레이션 루프 =====
model.train()
t = 0
while t < T:
    selected_nodes = []
    for i in range(N):
        if t >= event_times[i] and train_mask_np[i]:
            selected_nodes.append(i)
            event_logs[i].append(t)
            event_times[i] += np.random.exponential(1/lambda_)

    t += time_step

    if selected_nodes:
        loss_val = train_poisson(selected_nodes)
        if loss_val is not None:
            print(f"[Time {t:.1f}] Poisson Loss: {loss_val:.4f}")

print("=== Poisson-based Training Complete ===")

# ✅ 6. 이제 200 epoch 학습 (풀-배치 방식, Best Test Accuracy 저장)
best_test_acc = 0
best_epoch = 0

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data.to(device))
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # 테스트 수행
    model.eval()
    with torch.no_grad():
        out = model(data.to(device))
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
        test_acc = correct / data.test_mask.sum().item()

    # 🔹 Test Accuracy가 가장 높은 순간 저장
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch  # 언제 최고였는지 저장

    if epoch % 10 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")

print(f"=== Training Complete ===")
print(f"Best Test Accuracy: {best_test_acc:.4f} (at Epoch {best_epoch})")


[Time 0.2] Poisson Loss: 1.7969
[Time 0.3] Poisson Loss: 1.7786
[Time 0.4] Poisson Loss: 1.7631
[Time 0.5] Poisson Loss: 1.7004
[Time 0.6] Poisson Loss: 1.6389
[Time 0.7] Poisson Loss: 1.5969
[Time 0.8] Poisson Loss: 1.5479
[Time 0.9] Poisson Loss: 1.4088
[Time 1.0] Poisson Loss: 1.3526
[Time 1.1] Poisson Loss: 1.3874
[Time 1.2] Poisson Loss: 1.3544
[Time 1.3] Poisson Loss: 1.2161
[Time 1.4] Poisson Loss: 1.2222
[Time 1.5] Poisson Loss: 1.2324
[Time 1.6] Poisson Loss: 1.1598
[Time 1.7] Poisson Loss: 1.0959
[Time 1.8] Poisson Loss: 1.0752
[Time 1.9] Poisson Loss: 0.9887
[Time 2.0] Poisson Loss: 1.0838
[Time 2.1] Poisson Loss: 0.9889
[Time 2.2] Poisson Loss: 0.9808
[Time 2.3] Poisson Loss: 0.9716
[Time 2.4] Poisson Loss: 0.9800
[Time 2.5] Poisson Loss: 1.0090
[Time 2.6] Poisson Loss: 0.9331
[Time 2.7] Poisson Loss: 0.9495
[Time 2.8] Poisson Loss: 0.9044
[Time 2.9] Poisson Loss: 0.9706
[Time 3.0] Poisson Loss: 0.8588
[Time 3.1] Poisson Loss: 0.8651
[Time 3.2] Poisson Loss: 0.8383
[Time 3.

In [88]:
import torch
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn import BatchNorm1d, Linear
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# ✅ 1. Citeseer 데이터셋 로드
dataset = Planetoid(root='data/', name='Citeseer')
data = dataset[0]

# ✅ 2. Train Mask 생성 (test_mask와 겹치지 않는 노드 1800개 선택)
test_mask = data.test_mask
non_test_indices = torch.where(~test_mask)[0]
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]

new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True
train_mask = new_train_mask  # 새 train_mask 적용

# ✅ 3. 3-layer GCN + Projection Layer 추가
class GCN(torch.nn.Module):
    def __init__(self, in_features, hidden_dim1, hidden_dim2, out_features):
        super().__init__()
        self.conv1 = GCNConv(in_features, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, out_features)

        self.bn1 = BatchNorm1d(hidden_dim1)
        self.bn2 = BatchNorm1d(hidden_dim2)

        # ✅ Projection Layer 추가 (입력 차원과 맞춤)
        self.projection = Linear(in_features, out_features)

        self.reset_parameters()

    def reset_parameters(self):
        init.xavier_uniform_(self.conv1.lin.weight)
        init.xavier_uniform_(self.conv2.lin.weight)
        init.xavier_uniform_(self.conv3.lin.weight)
        init.xavier_uniform_(self.projection.weight)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # ✅ Residual Connection을 위한 원본 특징 저장
        x_res = self.projection(x)  # 입력 차원과 출력 차원을 맞춰줌

        # ✅ 첫 번째 GCN 레이어
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # ✅ 두 번째 GCN 레이어
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # ✅ 세 번째 GCN 레이어 (출력)
        x = self.conv3(x, edge_index)

        # ✅ Residual Connection 추가
        x += x_res  # 원래 입력 특징을 더해줌 (차원 맞춤)

        return F.log_softmax(x, dim=1)

# ✅ 4. 모델 및 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(
    in_features=dataset.num_features, 
    hidden_dim1=64, 
    hidden_dim2=32, 
    out_features=dataset.num_classes
).to(device)

data = data.to(device)
train_mask = train_mask.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-4)
criterion = torch.nn.NLLLoss()

# ✅ 5. 학습 루프
best_test_acc = 0
best_epoch = 0

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # 테스트 수행
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
        test_acc = correct / data.test_mask.sum().item()

    # 🔹 Best Test Accuracy 저장
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch

    if epoch % 10 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")

print(f"=== Training Complete ===")
print(f"Best Test Accuracy: {best_test_acc:.4f} (at Epoch {best_epoch})")


[Epoch 000] Loss: 2.1425 | Test Acc: 0.3440
[Epoch 010] Loss: 0.5542 | Test Acc: 0.7180
[Epoch 020] Loss: 0.3326 | Test Acc: 0.7640
[Epoch 030] Loss: 0.2069 | Test Acc: 0.7600
[Epoch 040] Loss: 0.1510 | Test Acc: 0.7660
[Epoch 050] Loss: 0.1123 | Test Acc: 0.7590
[Epoch 060] Loss: 0.0867 | Test Acc: 0.7560
[Epoch 070] Loss: 0.0675 | Test Acc: 0.7540
[Epoch 080] Loss: 0.0588 | Test Acc: 0.7520
[Epoch 090] Loss: 0.0487 | Test Acc: 0.7510
[Epoch 100] Loss: 0.0437 | Test Acc: 0.7470
[Epoch 110] Loss: 0.0386 | Test Acc: 0.7470
[Epoch 120] Loss: 0.0338 | Test Acc: 0.7470
[Epoch 130] Loss: 0.0324 | Test Acc: 0.7440
[Epoch 140] Loss: 0.0307 | Test Acc: 0.7450
[Epoch 150] Loss: 0.0256 | Test Acc: 0.7470
[Epoch 160] Loss: 0.0252 | Test Acc: 0.7440
[Epoch 170] Loss: 0.0241 | Test Acc: 0.7400
[Epoch 180] Loss: 0.0213 | Test Acc: 0.7390
[Epoch 190] Loss: 0.0196 | Test Acc: 0.7400
=== Training Complete ===
Best Test Accuracy: 0.7700 (at Epoch 17)


### 이번에는 Cora

In [7]:
dataset = Planetoid(root='data/', name='Cora')
data = dataset[0]

#############################################
# 2) (선택) 원하는 방식으로 train_mask 설정
#    - 예: 테스트셋 제외 후 1000개 노드 골라서 train_mask 구성 등
#    - 여기서는 간단히 data.train_mask 사용
#############################################
test_mask = data.test_mask

# 3. test_mask가 False인 노드들(= non-test 노드)만 골라낸 인덱스
non_test_indices = torch.where(~test_mask)[0]  # ~test_mask: test_mask가 False인 노드

# 4. non_test_indices 중에서 랜덤하게 1000개 선택
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1200]]

# 5. 새로운 train_mask 생성
new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True

# 6. data.train_mask에 할당
train_mask = new_train_mask

# 이제 data.train_mask에는 test set과 겹치지 않는 1000개의 노드만 True로 설정됨
print("Train set size:", train_mask.sum().item())
print("Overlap with test set:", (train_mask & data.test_mask).sum().item())  # 0이어야 정상

#############################################
# 3) GCN 모델 정의
#############################################
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data, selected_nodes=None):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.log_softmax(x, dim=1)

        # 특정 노드만 반환 (Poisson 시뮬레이션 시 사용)
        if selected_nodes is not None and len(selected_nodes) > 0:
            selected_nodes = torch.tensor(selected_nodes, dtype=torch.long, device=x.device)
            return x[selected_nodes]  
        return x

#############################################
# 4) 모델 및 옵티마이저 정의
#############################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss()

#############################################
# 5) (시나리오 B) 먼저 Poisson 시뮬레이션 기반 학습
#############################################
N = data.num_nodes
lambda_ = 0.5
T = 40  # 총 시뮬레이션 시간
time_step = 0.1

# 각 노드별 첫 이벤트 시간 (지수분포)
event_times = [np.random.exponential(1/lambda_) for _ in range(N)]
event_logs = [[] for _ in range(N)]

# numpy 형태의 train_mask (True/False)
train_mask_np = train_mask.cpu().numpy()

def train_poisson(selected_nodes):
    if len(selected_nodes) == 0:
        return None
    model.train()
    optimizer.zero_grad()

    out = model(data.to(device), selected_nodes)
    labels = data.y[selected_nodes].to(device)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# ===== Poisson 시뮬레이션 루프 =====
model.train()
t = 0
while t < T:
    selected_nodes = []
    for i in range(N):
        # Poisson 이벤트 발생 & train_mask
        if t >= event_times[i] and train_mask_np[i]:
            selected_nodes.append(i)
            event_logs[i].append(t)
            # 다음 이벤트 시간
            event_times[i] += np.random.exponential(1/lambda_)

    t += time_step

    # 학습
    if selected_nodes:
        loss_val = train_poisson(selected_nodes)
        if loss_val is not None:
            print(f"[Time {t:.1f}] Poisson Loss: {loss_val:.4f}")

print("=== Poisson-based Training Complete ===")


#############################################
# 6) 이제 200 epoch 학습(풀-배치 방식)
#############################################
model.train()
for epoch in range(1, 201):
    optimizer.zero_grad()
    out = model(data.to(device))  # 전체 노드에 대한 forward
    loss = criterion(out[train_mask], data.y[train_mask].to(device))
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss.item():.4f}")

print("=== Final 200 Epoch Training Complete ===")

def test():
    model.eval()
    with torch.no_grad():
        out = model(data.to(device))  # 모든 노드에 대한 예측
        pred = out.argmax(dim=1)  # 가장 높은 확률을 가진 클래스를 예측
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()  # 정확히 맞춘 개수
        acc = correct / data.test_mask.sum().item()  # 정확도 계산
    return acc

# 🔹 테스트 실행
test_acc = test()
print(f"Test Accuracy: {test_acc:.4f}")


Train set size: 1200
Overlap with test set: 0
[Time 0.2] Poisson Loss: 1.9569
[Time 0.3] Poisson Loss: 1.8972
[Time 0.4] Poisson Loss: 1.8401
[Time 0.5] Poisson Loss: 1.7912
[Time 0.6] Poisson Loss: 1.7283
[Time 0.7] Poisson Loss: 1.6761
[Time 0.8] Poisson Loss: 1.4945
[Time 0.9] Poisson Loss: 1.4464
[Time 1.0] Poisson Loss: 1.4060
[Time 1.1] Poisson Loss: 1.4227
[Time 1.2] Poisson Loss: 1.2878
[Time 1.3] Poisson Loss: 1.2073
[Time 1.4] Poisson Loss: 1.2002
[Time 1.5] Poisson Loss: 1.0866
[Time 1.6] Poisson Loss: 1.0475
[Time 1.7] Poisson Loss: 1.1163
[Time 1.8] Poisson Loss: 0.9510
[Time 1.9] Poisson Loss: 0.9116
[Time 2.0] Poisson Loss: 0.8285
[Time 2.1] Poisson Loss: 0.8101
[Time 2.2] Poisson Loss: 0.8257
[Time 2.3] Poisson Loss: 0.7444
[Time 2.4] Poisson Loss: 0.7504
[Time 2.5] Poisson Loss: 0.6202
[Time 2.6] Poisson Loss: 0.7280
[Time 2.7] Poisson Loss: 0.7471
[Time 2.8] Poisson Loss: 0.7894
[Time 2.9] Poisson Loss: 0.6880
[Time 3.0] Poisson Loss: 0.6304
[Time 3.1] Poisson Loss: 0

### 4 layers

In [120]:
import torch
import torch.nn.functional as F
import torch.nn.init as init
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# ✅ 1. Cora 데이터셋 로드
dataset = Planetoid(root='data/', name='Cora')
data = dataset[0]

# ✅ 2. Train Mask 생성 (test_mask와 겹치지 않는 1800개 노드 선택)
test_mask = data.test_mask
non_test_indices = torch.where(~test_mask)[0]
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1200]]

new_train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
new_train_mask[train_indices] = True
train_mask = new_train_mask  # 새 train_mask 적용

# ✅ 3. 4-layer GCN 모델 정의 (BatchNorm 제거)
class GCN(torch.nn.Module):
    def __init__(self, in_features, hidden_dim1, hidden_dim2, hidden_dim3, out_features):
        super().__init__()
        self.conv1 = GCNConv(in_features, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, hidden_dim3)
        self.conv4 = GCNConv(hidden_dim3, out_features)

        self.reset_parameters()

    def reset_parameters(self):
        init.xavier_uniform_(self.conv1.lin.weight)
        init.xavier_uniform_(self.conv2.lin.weight)
        init.xavier_uniform_(self.conv3.lin.weight)
        init.xavier_uniform_(self.conv4.lin.weight)

    def forward(self, data, selected_nodes=None):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        x = self.conv4(x, edge_index)
        x = F.log_softmax(x, dim=1)

        if selected_nodes is not None and len(selected_nodes) > 0:
            selected_nodes = torch.tensor(selected_nodes, dtype=torch.long, device=x.device)
            return x[selected_nodes]  
        return x

# ✅ 4. 모델 및 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(
    in_features=dataset.num_features, 
    hidden_dim1=64, 
    hidden_dim2=32, 
    hidden_dim3=16, 
    out_features=dataset.num_classes
).to(device)

data = data.to(device)
train_mask = train_mask.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
criterion = torch.nn.NLLLoss()

# ✅ 5. Poisson 시뮬레이션 기반 학습
N = data.num_nodes
lambda_ = 0.5
T = 10  # 총 시뮬레이션 시간
time_step = 0.1

# 각 노드별 첫 이벤트 시간 (지수분포)
event_times = [np.random.exponential(1/lambda_) for _ in range(N)]
event_logs = [[] for _ in range(N)]

# numpy 형태의 train_mask (True/False)
train_mask_np = train_mask.cpu().numpy()

def train_poisson(selected_nodes):
    if len(selected_nodes) == 0:
        return None
    model.train()
    optimizer.zero_grad()

    out = model(data.to(device), selected_nodes)
    labels = data.y[selected_nodes].to(device)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# ===== Poisson 시뮬레이션 루프 =====
model.train()
t = 0
while t < T:
    selected_nodes = []
    for i in range(N):
        if t >= event_times[i] and train_mask_np[i]:
            selected_nodes.append(i)
            event_logs[i].append(t)
            event_times[i] += np.random.exponential(1/lambda_)

    t += time_step

    if selected_nodes:
        loss_val = train_poisson(selected_nodes)
        if loss_val is not None:
            print(f"[Time {t:.1f}] Poisson Loss: {loss_val:.4f}")

print("=== Poisson-based Training Complete ===")

# ✅ 6. 이제 200 epoch 학습 (풀-배치 방식, Best Test Accuracy 저장)
best_test_acc = 0
best_epoch = 0

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data.to(device))
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # 테스트 수행
    model.eval()
    with torch.no_grad():
        out = model(data.to(device))
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
        test_acc = correct / data.test_mask.sum().item()

    # 🔹 Test Accuracy가 가장 높은 순간 저장
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch  # 언제 최고였는지 저장

    if epoch % 10 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")

print(f"=== Training Complete ===")
print(f"Best Test Accuracy: {best_test_acc:.4f} (at Epoch {best_epoch})")


[Time 0.2] Poisson Loss: 1.9273
[Time 0.3] Poisson Loss: 1.8685
[Time 0.4] Poisson Loss: 1.7790
[Time 0.5] Poisson Loss: 1.7411
[Time 0.6] Poisson Loss: 1.6240
[Time 0.7] Poisson Loss: 1.5658
[Time 0.8] Poisson Loss: 1.4673
[Time 0.9] Poisson Loss: 1.5078
[Time 1.0] Poisson Loss: 1.2359
[Time 1.1] Poisson Loss: 1.3279
[Time 1.2] Poisson Loss: 1.3021
[Time 1.3] Poisson Loss: 1.1882
[Time 1.4] Poisson Loss: 1.0880
[Time 1.5] Poisson Loss: 0.8803
[Time 1.6] Poisson Loss: 0.8957
[Time 1.7] Poisson Loss: 0.9983
[Time 1.8] Poisson Loss: 0.9296
[Time 1.9] Poisson Loss: 0.9022
[Time 2.0] Poisson Loss: 0.8067
[Time 2.1] Poisson Loss: 0.8207
[Time 2.2] Poisson Loss: 0.8101
[Time 2.3] Poisson Loss: 0.6444
[Time 2.4] Poisson Loss: 0.5908
[Time 2.5] Poisson Loss: 0.6608
[Time 2.6] Poisson Loss: 0.6179
[Time 2.7] Poisson Loss: 0.6301
[Time 2.8] Poisson Loss: 0.5700
[Time 2.9] Poisson Loss: 0.5816
[Time 3.0] Poisson Loss: 0.4714
[Time 3.1] Poisson Loss: 0.4689
[Time 3.2] Poisson Loss: 0.3193
[Time 3.

## 2. train from baseline - about 140 train sets

In [17]:
dataset = Planetoid(root='data/', name='cora')
data = dataset[0]

#############################################
# 2) (선택) 원하는 방식으로 train_mask 설정
#    - 예: 테스트셋 제외 후 1000개 노드 골라서 train_mask 구성 등
#    - 여기서는 간단히 data.train_mask 사용
#############################################
train_mask = data.train_mask

#############################################
# 3) GCN 모델 정의
#############################################
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data, selected_nodes=None):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.log_softmax(x, dim=1)

        # 특정 노드만 반환 (Poisson 시뮬레이션 시 사용)
        if selected_nodes is not None and len(selected_nodes) > 0:
            selected_nodes = torch.tensor(selected_nodes, dtype=torch.long, device=x.device)
            return x[selected_nodes]  
        return x

#############################################
# 4) 모델 및 옵티마이저 정의
#############################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss()

#############################################
# 5) (시나리오 B) 먼저 Poisson 시뮬레이션 기반 학습
#############################################
N = data.num_nodes
lambda_ = 0.5
T = 60  # 총 시뮬레이션 시간
time_step = 0.3

# 각 노드별 첫 이벤트 시간 (지수분포)
event_times = [np.random.exponential(1/lambda_) for _ in range(N)]
event_logs = [[] for _ in range(N)]

# numpy 형태의 train_mask (True/False)
train_mask_np = train_mask.cpu().numpy()

def train_poisson(selected_nodes):
    if len(selected_nodes) == 0:
        return None
    model.train()
    optimizer.zero_grad()

    out = model(data.to(device), selected_nodes)
    labels = data.y[selected_nodes].to(device)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# ===== Poisson 시뮬레이션 루프 =====
model.train()
t = 0
while t < T:
    selected_nodes = []
    for i in range(N):
        # Poisson 이벤트 발생 & train_mask
        if t >= event_times[i] and train_mask_np[i]:
            selected_nodes.append(i)
            event_logs[i].append(t)
            # 다음 이벤트 시간
            event_times[i] += np.random.exponential(1/lambda_)

    t += time_step

    # 학습
    if selected_nodes:
        loss_val = train_poisson(selected_nodes)
        if loss_val is not None:
            print(f"[Time {t:.1f}] Poisson Loss: {loss_val:.4f}")

print("=== Poisson-based Training Complete ===")


#############################################
# 6) 이제 200 epoch 학습(풀-배치 방식)
#############################################
model.train()
for epoch in range(1, 201):
    optimizer.zero_grad()
    out = model(data.to(device))  # 전체 노드에 대한 forward
    loss = criterion(out[train_mask], data.y[train_mask].to(device))
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss.item():.4f}")

print("=== Final 200 Epoch Training Complete ===")

[Time 0.6] Poisson Loss: 1.9592
[Time 0.9] Poisson Loss: 1.8939
[Time 1.2] Poisson Loss: 1.8845
[Time 1.5] Poisson Loss: 1.7842
[Time 1.8] Poisson Loss: 1.8316
[Time 2.1] Poisson Loss: 1.5880
[Time 2.4] Poisson Loss: 1.5230
[Time 2.7] Poisson Loss: 1.5371
[Time 3.0] Poisson Loss: 1.3999
[Time 3.3] Poisson Loss: 1.3294
[Time 3.6] Poisson Loss: 1.1986
[Time 3.9] Poisson Loss: 1.1602
[Time 4.2] Poisson Loss: 1.2492
[Time 4.5] Poisson Loss: 1.0860
[Time 4.8] Poisson Loss: 0.9923
[Time 5.1] Poisson Loss: 1.0043
[Time 5.4] Poisson Loss: 1.0940
[Time 5.7] Poisson Loss: 0.9376
[Time 6.0] Poisson Loss: 0.7580
[Time 6.3] Poisson Loss: 0.7760
[Time 6.6] Poisson Loss: 0.9895
[Time 6.9] Poisson Loss: 0.8650
[Time 7.2] Poisson Loss: 0.6895
[Time 7.5] Poisson Loss: 0.6208
[Time 7.8] Poisson Loss: 0.5445
[Time 8.1] Poisson Loss: 0.7142
[Time 8.4] Poisson Loss: 0.8390
[Time 8.7] Poisson Loss: 0.6115
[Time 9.0] Poisson Loss: 0.5700
[Time 9.3] Poisson Loss: 0.5443
[Time 9.6] Poisson Loss: 0.4532
[Time 9.

In [18]:
def test():
    model.eval()
    with torch.no_grad():
        out = model(data.to(device))  # 모든 노드에 대한 예측
        pred = out.argmax(dim=1)  # 가장 높은 확률을 가진 클래스를 예측
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()  # 정확히 맞춘 개수
        acc = correct / data.test_mask.sum().item()  # 정확도 계산
    return acc

# 🔹 테스트 실행
test_acc = test()
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.8160


# 2. Oversmoothing 관련


In [9]:
import torch
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# ✅ 1. Citeseer 데이터셋 로드
dataset = Planetoid(root='data/', name='Citeseer')
data = dataset[0]

# ✅ 2. Train Mask 생성 (test_mask와 겹치지 않는 1800개 노드 선택)
test_mask = data.test_mask
non_test_indices = torch.where(~test_mask)[0]
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]

train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[train_indices] = True

# ✅ 3. 16-layer GCN 모델 (Residual Connection 없이 그냥 쌓음)
class DeepGCN(torch.nn.Module):
    def __init__(self, in_features, hidden_dim, out_features, num_layers=32):
        super().__init__()
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()
        
        # 첫 번째 레이어
        self.convs.append(GCNConv(in_features, hidden_dim))
        
        # 중간 레이어 (전부 hidden_dim 유지)
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        
        # 마지막 레이어
        self.convs.append(GCNConv(hidden_dim, out_features))
        
    def forward(self, data, selected_nodes=None):
        x, edge_index = data.x, data.edge_index

        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < self.num_layers - 1:  # 마지막 레이어는 활성화 X
                x = F.relu(x)
                x = F.dropout(x, p=0.2, training=self.training)

        x = F.log_softmax(x, dim=1)

        if selected_nodes is not None and len(selected_nodes) > 0:
            selected_nodes = torch.tensor(selected_nodes, dtype=torch.long, device=x.device)
            return x[selected_nodes]  
        return x

# ✅ 4. 모델 및 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepGCN(
    in_features=dataset.num_features, 
    hidden_dim=64, 
    out_features=dataset.num_classes,
    num_layers=16
).to(device)

data = data.to(device)
train_mask = train_mask.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
criterion = torch.nn.NLLLoss()

# ✅ 5. Poisson 시뮬레이션 기반 학습
N = data.num_nodes
lambda_ = 0.5
T = 6  # 총 시뮬레이션 시간
time_step = 0.1

# 각 노드별 첫 이벤트 시간 (지수분포)
event_times = [np.random.exponential(1/lambda_) for _ in range(N)]
event_logs = [[] for _ in range(N)]

# numpy 형태의 train_mask (True/False)
train_mask_np = train_mask.cpu().numpy()

def train_poisson(selected_nodes):
    if len(selected_nodes) == 0:
        return None
    model.train()
    optimizer.zero_grad()

    out = model(data, selected_nodes)
    labels = data.y[selected_nodes].to(device)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# ===== Poisson 시뮬레이션 루프 =====
model.train()
t = 0
while t < T:
    selected_nodes = []
    for i in range(N):
        if t >= event_times[i] and train_mask_np[i]:
            selected_nodes.append(i)
            event_logs[i].append(t)
            event_times[i] += np.random.exponential(1/lambda_)

    t += time_step

    if selected_nodes:
        loss_val = train_poisson(selected_nodes)
        if loss_val is not None:
            print(f"[Time {t:.1f}] Poisson Loss: {loss_val:.4f}")

print("=== Poisson-based Training Complete ===")

# ✅ 6. 이제 200 epoch 학습 (풀-배치 방식, Best Test Accuracy 저장)
best_test_acc = 0
best_epoch = 0

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # 테스트 수행
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
        test_acc = correct / data.test_mask.sum().item()

    # 🔹 Test Accuracy가 가장 높은 순간 저장
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch

    if epoch % 10 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")

print(f"=== Training Complete ===")
print(f"Best Test Accuracy: {best_test_acc:.4f} (at Epoch {best_epoch})")


[Time 0.2] Poisson Loss: 1.7918
[Time 0.3] Poisson Loss: 1.7894
[Time 0.4] Poisson Loss: 1.7833
[Time 0.5] Poisson Loss: 1.7752
[Time 0.6] Poisson Loss: 1.7608
[Time 0.7] Poisson Loss: 1.7322
[Time 0.8] Poisson Loss: 1.6344
[Time 0.9] Poisson Loss: 1.5654
[Time 1.0] Poisson Loss: 1.6932
[Time 1.1] Poisson Loss: 1.5506
[Time 1.2] Poisson Loss: 1.6668
[Time 1.3] Poisson Loss: 1.5483
[Time 1.4] Poisson Loss: 1.5329
[Time 1.5] Poisson Loss: 1.6460
[Time 1.6] Poisson Loss: 1.4995
[Time 1.7] Poisson Loss: 1.5557
[Time 1.8] Poisson Loss: 1.4877
[Time 1.9] Poisson Loss: 1.5330
[Time 2.0] Poisson Loss: 1.4929
[Time 2.1] Poisson Loss: 1.6411
[Time 2.2] Poisson Loss: 1.5707
[Time 2.3] Poisson Loss: 1.5276
[Time 2.4] Poisson Loss: 1.4577
[Time 2.5] Poisson Loss: 1.4895
[Time 2.6] Poisson Loss: 1.4629
[Time 2.7] Poisson Loss: 1.4908
[Time 2.8] Poisson Loss: 1.5677
[Time 2.9] Poisson Loss: 1.4704
[Time 3.0] Poisson Loss: 1.5152
[Time 3.1] Poisson Loss: 1.5553
[Time 3.2] Poisson Loss: 1.4585
[Time 3.

In [10]:
import torch
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# ✅ 데이터셋 로드
dataset = Planetoid(root='data/', name='Citeseer')
data = dataset[0]

# ✅ Train Mask 생성
test_mask = data.test_mask
non_test_indices = torch.where(~test_mask)[0]
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[train_indices] = True

# ✅ 16-layer GCN 모델 (Residual Connection 포함)
class DeepGCN(torch.nn.Module):
    def __init__(self, in_features, hidden_dim, out_features, num_layers=16):
        super().__init__()
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()
        
        # 첫 번째 레이어
        self.convs.append(GCNConv(in_features, hidden_dim))
        
        # 중간 레이어 (Residual 적용)
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        
        # 마지막 레이어
        self.convs.append(GCNConv(hidden_dim, out_features))
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        identity = x  # Residual을 위해 원본 x 저장

        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            
            if i < self.num_layers - 1:  # 마지막 레이어에서는 활성화 X
                x = F.relu(x)
                x = F.dropout(x, p=0.2, training=self.training)
                
                # 🔹 Residual Connection (매 2개 레이어마다 추가)
                if i % 2 == 1:
                    x = x + identity  # 원본 x를 더함
                    identity = x  # 업데이트된 x를 새로운 identity로 저장

        return F.log_softmax(x, dim=1)

# ✅ 모델 및 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepGCN(
    in_features=dataset.num_features, 
    hidden_dim=64, 
    out_features=dataset.num_classes,
    num_layers=16  # 16개 레이어 사용
).to(device)

data = data.to(device)
train_mask = train_mask.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
criterion = torch.nn.NLLLoss()

# ✅ 학습 루프
best_test_acc = 0
best_epoch = 0

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # 🔹 테스트 수행
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
        test_acc = correct / data.test_mask.sum().item()

    # 🔹 최고 정확도 저장
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch

    if epoch % 10 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")

print(f"=== Training Complete ===")
print(f"Best Test Accuracy: {best_test_acc:.4f} (at Epoch {best_epoch})")


RuntimeError: The size of tensor a (64) must match the size of tensor b (3703) at non-singleton dimension 1

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# ✅ 데이터셋 로드
dataset = Planetoid(root='data/', name='Citeseer')
data = dataset[0]

# ✅ Train Mask 생성
test_mask = data.test_mask
non_test_indices = torch.where(~test_mask)[0]
train_indices = non_test_indices[torch.randperm(len(non_test_indices))[:1800]]
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[train_indices] = True

# ✅ 16-layer GCN 모델 (Residual Connection 없음)
# class DeepGCN(torch.nn.Module):
#     def __init__(self, in_features, hidden_dim, out_features, num_layers=64):
#         super().__init__()
#         self.num_layers = num_layers
#         self.convs = torch.nn.ModuleList()
        
#         # 첫 번째 레이어
#         self.convs.append(GCNConv(in_features, hidden_dim))
        
#         # 중간 14개 레이어 (모두 hidden_dim 유지)
#         for _ in range(num_layers - 2):
#             self.convs.append(GCNConv(hidden_dim, hidden_dim))
        
#         # 마지막 레이어
#         self.convs.append(GCNConv(hidden_dim, out_features))
        
#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index

#         for i, conv in enumerate(self.convs):
#             x = conv(x, edge_index)
#             if i < self.num_layers - 1:  # 마지막 레이어는 활성화 X
#                 x = F.relu(x)
#                 x = F.dropout(x, p=0.2, training=self.training)

#         return F.log_softmax(x, dim=1)

class DeepGCN(torch.nn.Module):
    def __init__(self, in_features, hidden_dim, out_features, num_layers=64):
        super().__init__()
        self.num_layers = num_layers
        # self.convs = torch.nn.ModuleList()
        
        # 첫 번째 레이어
        modules = []
        modules.extend([GCNConv(in_features, hidden_dim), nn.ReLU()])
        for _ in range(num_layers - 2):
            modules.append([GCNConv(hidden_dim, hidden_dim), nn.ReLU()])
        modules.append(GCNConv(hidden_dim, out_features)

        self.layers = nn.Sequential(*modules)

    # https://pytorch-geometric.readthedocs.io/en/2.5.1/modules/nn.html 참고
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < self.num_layers - 1:  # 마지막 레이어는 활성화 X
                x = F.relu(x)
                x = F.dropout(x, p=0.2, training=self.training)

        return F.log_softmax(x, dim=1)

# ✅ 모델 및 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepGCN(
    in_features=dataset.num_features, 
    hidden_dim=64, 
    out_features=dataset.num_classes,
    num_layers=16
).to(device)

data = data.to(device)
train_mask = train_mask.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
criterion = torch.nn.NLLLoss()

# ✅ 학습 루프
best_test_acc = 0
best_epoch = 0

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # 🔹 테스트 수행
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
        test_acc = correct / data.test_mask.sum().item()

    # 🔹 최고 정확도 저장
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch

    if epoch % 10 == 0:
        print(f"[Epoch {epoch:03d}] Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")

print(f"=== Training Complete ===")
print(f"Best Test Accuracy: {best_test_acc:.4f} (at Epoch {best_epoch})")


[Epoch 000] Loss: 1.7918 | Test Acc: 0.2310
[Epoch 010] Loss: 1.6079 | Test Acc: 0.2960
[Epoch 020] Loss: 1.4406 | Test Acc: 0.3860
[Epoch 030] Loss: 1.3153 | Test Acc: 0.4280
[Epoch 040] Loss: 1.2483 | Test Acc: 0.4500
[Epoch 050] Loss: 1.1886 | Test Acc: 0.4470
[Epoch 060] Loss: 1.1633 | Test Acc: 0.5200
[Epoch 070] Loss: 1.0384 | Test Acc: 0.5950
[Epoch 080] Loss: 0.9452 | Test Acc: 0.6010
[Epoch 090] Loss: 0.8413 | Test Acc: 0.6980
[Epoch 100] Loss: 0.7850 | Test Acc: 0.7040
[Epoch 110] Loss: 0.7250 | Test Acc: 0.6940
[Epoch 120] Loss: 0.6812 | Test Acc: 0.7190
[Epoch 130] Loss: 0.6799 | Test Acc: 0.7040
[Epoch 140] Loss: 0.6682 | Test Acc: 0.7130
[Epoch 150] Loss: 0.6560 | Test Acc: 0.7240
[Epoch 160] Loss: 0.6376 | Test Acc: 0.7200
[Epoch 170] Loss: 0.6370 | Test Acc: 0.7300
[Epoch 180] Loss: 0.6272 | Test Acc: 0.7250
[Epoch 190] Loss: 0.6131 | Test Acc: 0.7340
=== Training Complete ===
Best Test Accuracy: 0.7400 (at Epoch 176)
